# SQL Package

Provides simple functionality to interact with a PostgreSQL server using Python classes.

**Overview of functionality:**
* Database(self, user, password, host, dbname, port)
    * properties
        * user
        * password
        * host
        * dbname
        * port
    * methods
        * create(name) x
        * connect()
        * drop(name)
* Table(self, dbname, table, schema)
    * accepts db properties
    * properties
        * connect() --> inherited
        * fetch_data(sql, con, parse_dates)
        * get_names()
        * format_names(char_dict)
        * update_names(names_dict)
        * add_columns(columns_list, type=None)
        * compare_column_order(dataframe)
        * match_columns(dataframe)
        * save_csv(data, local_path, match_column_order=True)
        * update_values(local_path, container_path)
        * update_types(types_dict)
        * close()

## Setup

In [101]:
import os
import sys
from pathlib import Path
#sys.path[0] = str(Path(__file__).resolve().parents[2]) # Set path for custom modules

# Set path for modules
sys.path[0] = '../'

from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

# SQL libraries
import psycopg2

# Set notebook display options
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [102]:
#sys.modules

In [325]:
class Database():
    
    # if modulename not in sys.modules: print...
    load_dotenv(find_dotenv());
    
    def __init__(self, user=None, password=None,
                 dbname=None, host=None, port=None):
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
        
        
        # Root directory
        self._root_dir = os.path.dirname(os.getcwd())
        #sys.path[0] = str(Path(__file__).resolve().parents[2])
        
    def _connect(self):

        """
        Connects to PostgreSQL database using psycopg2 driver. Same
        arguments as psycopg2.connect().

        Params
        --------
        dbname
        user
        password
        host
        port
        connect_timeout
        """

        try:
            con = psycopg2.connect(dbname=self.dbname,
                                   user=self.user,
                                   password=self.password,
                                    host=self.host, 
                                    port=self.port,
                                  connect_timeout=3)            

        except Exception as e:
            print('Error:\n', e)
            return None


        return con        

In [323]:
db = Database()

Connected as user "postgres" to database "permits" on http://localhost:5432.


In [351]:
class Table(Database):
    def __init__(self, user=None, password=None, dbname=None, host=None, port=None, table=None):
        super().__init__(user, password, dbname, host, port)
        
        self.table = table
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
    
    # Connect to database
    def _connect(self):
        return super(Table, self)._connect()
    
    # Fetch data from sql query
    def fetch_data(self, sql, coerce_float=False, parse_dates=None):
        
        con = self._connect()
        
        # Fetch fresh data
        data = pd.read_sql_query(sql=sql, con=con, coerce_float=coerce_float, parse_dates=parse_dates)

        # Replace None with np.nan
        data.fillna(np.nan, inplace=True)
        
        # Close db connection
        con.close()

        return data
    
    # Get names of column
    def get_names(self):
        
        con = self._connect()
        
        # Specific query to retrieve table names
        sql = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{}'".format(self.table)
        etl = pd.read_sql_query(sql, con)
        columns = etl['column_name']
    
        return columns

    # Standardize column names using dictionary of character replacements
    def reformat_names(self, replace_map, execute=False):
        
        series = self.get_names()
        
        def replace_chars(text):
            for oldchar, newchar in replace_map.items():
                text = text.replace(oldchar, newchar).lower()
            return text

        if execute:
            print("Replaced in database.")
        
        return series.apply(replace_chars)
    
    # Update column names in db table
    def _update_names(old_columns, new_columns, db_table, path, run=False):
        
        old_columns = 
        
        new_columns = 
        
        con = self._connect();
    
        sql = 'ALTER TABLE {} '.format(db_table) + 'RENAME "{old_name}" to {new_name};'

        sql_query = []

        for idx, name in old_columns.iteritems():
            sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))

        update_names = '\n'.join(sql_query)

        # replace with path
        with open(path, 'w') as text:
            text.write(update_names)

        # Update db is desired
        if run:
            try:
                cur = con.cursor()
                print("Reading...")
                sql_file = open(path, 'r')
                print('Executing update query on table "{}"...'.format(db_table))
                cur.execute(sql_file.read())
                con.commit()
                cur.close()
                print("Table is updated.")
            except Exception as e:
                conn.rollback()
                print('Error:\n', e)

In [348]:
permits = Table(table="permits_raw")
data = permits.fetch_data(sql="SELECT * FROM permits_raw LIMIT 10;")

In [349]:
permits.get_names()[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [350]:
# Map of character replacements
replace_map = {' ': '_', '-': '_', '#': 'No', '/': '_', 
               '.': '', '(': '', ')': '', "'": ''}

permits.reformat_names(replace_map)[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object